In [ ]:
%pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
%pip install bitsandbytes einops wandb -Uqqq

In [ ]:
import torch
import glob
import os
import pandas as pd
import numpy as np
import re
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
from datasets import Dataset
import matplotlib.pyplot as plt
import random

In [ ]:
TEXT_PATH = "drive/MyDrive/LoveCraft_tinyllama/sentences.txt"
with open(TEXT_PATH) as file:
  stories = file.read()
  file.close()

stories

In [ ]:
# List of all unique characters
print(' '.join(sorted(set(stories))))

In [ ]:
remove_list = [",","."]
cleaned = stories
for string in remove_list:
    cleaned = stories.replace(string, " ")

In [ ]:
cleaned

In [ ]:
print(' '.join(sorted(set(cleaned))))

In [ ]:
# Setting aside a portion for training the model and a portion for testing the data to prevent
# the model from overfitting to the data it is tested on
split_point = int(len(stories)*0.95)
train_data = stories[:split_point]
test_data = stories[split_point:]
train_data_seg = []
eval_data_seg = []

for i in range(0, len(train_data), 100):
  text = train_data[i:min(i+100, len(train_data))]
  train_data_seg.append(text)

for i in range(0, len(test_data), 100):
  text = test_data[i:min(i+100, len(test_data))]
  eval_data_seg.append(text)

random.shuffle(train_data_seg)
train_data_seg = Dataset.from_dict({'text':train_data_seg})
eval_data_seg = Dataset.from_dict({"text": eval_data_seg})
print(len(train_data_seg))

In [ ]:
len(test_data)

In [ ]:
# You will need to create a Hugging Face account if you do not have one,
# and then generate a write token to enter in the widget below
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_name = "PY007/TinyLlama-1.1B-step-50K-105b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Generating lyrics with the base model. The repetition penalty in the generation config prevents the model from continually repeating the same string.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def generate_lyrics(query, model):
    encoding = tokenizer(query, return_tensors="pt").to(device)
    generation_config = GenerationConfig(max_new_tokens=500, pad_token_id = tokenizer.eos_token_id,repetition_penalty=1.3, eos_token_id = tokenizer.eos_token_id)
    outputs = model.generate(input_ids=encoding.input_ids, generation_config=generation_config)
    text_output = tokenizer.decode(outputs[0],skip_special_tokens=True)
    print('INPUT\n', query, '\n\nOUTPUT\n', text_output[len(query):])
generate_lyrics(test_data[:int(len(test_data)*0.05)], model)

In [ ]:
# Setting arguments for low-rank adaptation

model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # The weight matrix is scaled by lora_alpha/lora_rank, so I set lora_alpha = lora_rank to remove scaling
lora_dropout = 0.05
lora_rank = 32

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM")

peft_model = get_peft_model(model, peft_config)

In [ ]:
output_dir = "SINEdowskY/tiny_llama_lovecraft" # Model repo on your hugging face account where you want to save your model
per_device_train_batch_size = 3
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_strategy="steps"
save_steps = 50
logging_steps = 10
learning_rate = 2e-3
max_grad_norm = 0.3 # Sets limit for gradient clipping
max_steps = 1000     # Number of training steps
warmup_ratio = 0.03 # Portion of steps used for learning_rate to warmup from 0
lr_scheduler_type = "cosine" # I chose cosine to avoid learning plateaus

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    evaluation_strategy="steps",
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
    report_to='none'
)

In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_data_seg,
    eval_dataset=eval_data_seg,
    peft_config=peft_config,
    max_seq_length=100,
    dataset_text_field='text',
    tokenizer=tokenizer,
    args=training_arguments
)
peft_model.config.use_cache = False

In [ ]:
trainer.train()

In [ ]:
train_loss_values = [el["loss"] for el in trainer.state.log_history[:-1]]
# Plot the training loss
plt.plot(train_loss_values, label="Training Loss")
plt.xlabel("Training Steps")
plt.ylabel("Loss")
plt.title("Training Loss Over Steps")
plt.legend()
plt.show()

In [ ]:
generate_lyrics(test_data[:int(len(test_data)*0.1)], model)